In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import math
from math import sqrt
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.pylab import rcParams
import matplotlib.gridspec as gridspec
from scipy.stats import zscore
import csv
import codecs
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%config InlineBackend.figure_format = 'retina'

In [2]:
data = []

with codecs.open('dataset/good.csv', encoding='utf-8', errors='ignore') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        data.append(row)
        
data1 = pd.DataFrame(data)
data1.shape

(4559, 99)

In [3]:
data = []

with codecs.open('dataset/bad.csv', encoding='utf-8', errors='ignore') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        data.append(row)
        
data2 = pd.DataFrame(data)
data2.shape

(135, 99)

In [12]:
data = []

with codecs.open('dataset/val.csv', encoding='utf-8', errors='ignore') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        data.append(row)
        
data3 = pd.DataFrame(data)
data3.shape

(1815, 99)

In [5]:
# 留下的特徵
all_name = [
#     'ShotSN',
#     'MoldClampedTime',
#     'MoldReleasedTime',
#     'MOID',
#     'ProdID',
    'Sensor2_MaxPressure',
    'Sensor2_SumStage1Pressure',
#     'Sensor2_MaxPressureTime',
    'Sensor2_AvgStage1PressureSlope',
    'Sensor2_AvgStage2PressureSlope',
    'Sensor2_MoldReleasePressure',
    'Sensor3_MaxPressure',
    'Sensor3_SumStage1Pressure',
#     'Sensor3_MaxPressureTime',
    'Sensor3_AvgStage1PressureSlope',
    'Sensor3_AvgStage2PressureSlope',
    'Sensor3_MoldReleasePressure',
    'Sensor4_MaxPressure',
    'Sensor4_SumStage1Pressure',
#     'Sensor4_MaxPressureTime',
    'Sensor4_AvgStage1PressureSlope',
    'Sensor4_AvgStage2PressureSlope',
    'Sensor4_MoldReleasePressure',
    'Sensor5_StartTemperature',
    'Sensor5_MaxTemperature',
    'Sensor5_AvgStage2TemperatureSlope',
    'Sensor5_MoldReleaseTemperature',
#     'Sensor1_Stage1StartTime',
#     'Sensor1_Stage1EndTime',
#     'Sensor1_Stage1Period',
    'Sensor1_AvgStage1PressureSlope',
    'Sensor1_SumStage1Pressure',
    'Sensor1_Stage1EndPressure',
#     'Sensor1_Stage2StartTime',
#     'Sensor1_Stage2EndTime',
#     'Sensor1_Stage2Period',
    'Sensor1_AvgStage2PressureSlope',
    'Sensor1_SumStage2Pressure',
#     'Sensor1_Stage3StartTime',
#     'Sensor1_Stage3EndTime',
#     'Sensor1_Stage3Period',
    'Sensor1_MaxStage3Pressure',
#     'Sensor1_MaxStage3PressureTime',
    'Sensor1_SumStage3Pressure',
#     'Sensor1_CycleTime',
    'Sensor1_SumPressure',
#     'Soft_MoldClampedCycleTime',
#     'Soft_MoldBasedCycleTime',
    'Soft_MoldShotInterval',
#     'Soft_MoldCompleteCycleTime',
    'Soft_DiffMaxPressure',
#     'Soft_DiffIncreasePressureTime',
    'Soft_DiffMoldReleasePressure',
    'Type',
]

In [6]:
data1.columns = data1.iloc[0]
data1 = data1.iloc[2:, :]
data1 = data1.reset_index(drop=True)
data1.head()

,ShotSN,MoldClampedTime,MoldReleasedTime,MOID,ProdID,Sensor2_MaxPressure,Sensor2_SumStage1Pressure,Sensor2_MaxPressureTime,Sensor2_AvgStage1PressureSlope,Sensor2_AvgStage2PressureSlope,Sensor2_MoldReleasePressure,Sensor3_MaxPressure,Sensor3_SumStage1Pressure,Sensor3_MaxPressureTime,Sensor3_AvgStage1PressureSlope,Sensor3_AvgStage2PressureSlope,Sensor3_MoldReleasePressure,Sensor4_MaxPressure,Sensor4_SumStage1Pressure,Sensor4_MaxPressureTime,Sensor4_AvgStage1PressureSlope,Sensor4_AvgStage2PressureSlope,Sensor4_MoldReleasePressure,Sensor5_StartTemperature,Sensor5_MaxTemperature,Sensor5_AvgStage2TemperatureSlope,Sensor5_MoldReleaseTemperature,Sensor1_Stage1StartTime,Sensor1_Stage1EndTime,Sensor1_Stage1Period,Sensor1_AvgStage1PressureSlope,Sensor1_SumStage1Pressure,Sensor1_Stage1EndPressure,Sensor1_Stage2StartTime,Sensor1_Stage2EndTime,Sensor1_Stage2Period,Sensor1_AvgStage2PressureSlope,Sensor1_SumStage2Pressure,Sensor1_Stage3StartTime,Sensor1_Stage3EndTime,Sensor1_Stage3Period,Sensor1_MaxStage3Pressure,Sensor1_MaxStage3PressureTime,Sensor1_SumStage3Pressure,Sensor1_CycleTime,Sensor1_SumPressure,Soft_MoldClampedCycleTime,Soft_MoldBasedCycleTime,Soft_MoldShotInterval,Soft_MoldCompleteCycleTime,Soft_DiffMaxPressure,Soft_DiffIncreasePressureTime,Soft_DiffMoldReleasePressure,OPCUA_CycleTime,OPCUA_MoldClampingTime,OPCUA_MoldReleasingTime,OPCUA_InjectionTime,OPCUA_VPChangeOverDelayTime,OPCUA_HoldingTime,OPCUA_PlastificationTime,OPCUA_VPChangeOverPosition,OPCUA_CusionVolume,OPCUA_EndHoldingPosition,OPCUA_EndPlastificationPosition,OPCUA_Nozzle1Temperature,OPCUA_Nozzle2Temperature,OPCUA_Nozzle3Temperature,OPCUA_Nozzle4Temperature,OPCUA_Nozzle5Temperature,OPCUA_IMMOilTemperature,OPCUA_FallMaterialTemperature,OPCUA_MaxInjectionPressure,OPCUA_VPChangeOverPressure,OPCUA_SumInjectionPressure,OPCUA_SumHoldingPressure,OPCUA_MaxCavityNearGatePressure,OPCUA_MaxCavityFarGatePressure,OPCUA_SumCavityPressure,OPCUA_SumMoldClampingForce,OPCUA_MaxMoldClampingForce,OPCUA_MaxBackPressure,OPCUA_DelayPlastificationTime,OPCUA_InMoldCoolingTemperature,OPCUA_OutMoldCoolingTemperature,OPCUA_MoldCoolingVolume,OPCUA_MoldReleasingPosition2,OPCUA_MoldReleasingPosition3,OPCUA_MoldReleasingPosition4,OPCUA_MoldReleasingPosition5,OPCUA_ScrewSuckBackPosition,OPCUA_ScrewRPM,OPCUA_PartWeight,OPCUA_GateClosingTime,OPCUA_MeltTemperature,OPCUA_MoldTemperature,StartShotSN,EndShotSN,GroupSN,Type
0,1,09:23.8,11:49.8,t11,#2-4^TOY,1.06,14.14879,48.305779,0.021944,-0.011339,0,0.99,9.154309,45.064541,0.021747,-0.010132,0.01,1.04,7.08935,42.935417,0.024223,-0.010319,0.02,0.48,1.98,-0.010371,0.7,5.582084,32.348705,26.766621,0.034745,562.98,1.44,32.348705,66.638178,34.289473,-0.037038,898.21,68.007075,82.950629,14.943554,0.22,68.05774,65.09,141.785707,1821.66,141.854845,150.930355,0,150.930355,0.07,3.241238,-0.01,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1,1,1,0
1,2,12:12.2,14:38.1,t11,#2-4^TOY,0.96,8.488392,46.251426,0.020756,-0.010049,0,1.12,10.489044,43.460442,0.025541,-0.010781,0.06,0.99,10.312204,46.70804,0.021196,-0.010413,0,0.54,1.98,-0.010136,0.73,5.426738,35.197341,29.770603,0.030567,644.65,1.43,35.197341,66.642614,31.445273,-0.041024,822.68,67.962057,82.828366,14.866309,0.23,68.01281,64.68,141.784266,1825.97,141.852433,150.900857,17.399743,168.3006,-0.16,2.790984,-0.06,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2,2,2,0
2,3,15:00.4,17:26.5,t11,#2-4^TOY,0.98,16.411448,53.499554,0.018318,-0.011088,0,1.02,7.885406,44.623369,0.020841,-0.010282,0.02,1.09,11.335504,46.196566,0.023595,-0.011182,0.02,0.43,1.98,-0.013864,0.27,5.880455,35.693914,29.813459,0.030523,650.36,1.43,35.693914,66.642759,30.948845,-0.035866,808.26,66.592006,67.200962,0.608956,1.12,66.592006,2.52,141.882993,1820.92,141.95093,150.975584,17.396936,168.37252,-0.04,8.876185,-0.02,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,3,3,3,0
3,4,17:48.9,20:14.9,t11,#2-4^TOY,1.09,10.686955,43.714665,0.024935,-0.010695,0.04,1.03,7.681343,42.801217,0.023598,-0.010294,0.01,1.02,11.518071,48.078887,0.020175,-0.010873,0,0.27,1.98,-0.01205

In [7]:
data2.columns = data2.iloc[0]
data2 = data2.iloc[2:, :]
data2 = data2.reset_index(drop=True)
data2.head()

,ShotSN,MoldClampedTime,MoldReleasedTime,MOID,ProdID,Sensor2_MaxPressure,Sensor2_SumStage1Pressure,Sensor2_MaxPressureTime,Sensor2_AvgStage1PressureSlope,Sensor2_AvgStage2PressureSlope,Sensor2_MoldReleasePressure,Sensor3_MaxPressure,Sensor3_SumStage1Pressure,Sensor3_MaxPressureTime,Sensor3_AvgStage1PressureSlope,Sensor3_AvgStage2PressureSlope,Sensor3_MoldReleasePressure,Sensor4_MaxPressure,Sensor4_SumStage1Pressure,Sensor4_MaxPressureTime,Sensor4_AvgStage1PressureSlope,Sensor4_AvgStage2PressureSlope,Sensor4_MoldReleasePressure,Sensor5_StartTemperature,Sensor5_MaxTemperature,Sensor5_AvgStage2TemperatureSlope,Sensor5_MoldReleaseTemperature,Sensor1_Stage1StartTime,Sensor1_Stage1EndTime,Sensor1_Stage1Period,Sensor1_AvgStage1PressureSlope,Sensor1_SumStage1Pressure,Sensor1_Stage1EndPressure,Sensor1_Stage2StartTime,Sensor1_Stage2EndTime,Sensor1_Stage2Period,Sensor1_AvgStage2PressureSlope,Sensor1_SumStage2Pressure,Sensor1_Stage3StartTime,Sensor1_Stage3EndTime,Sensor1_Stage3Period,Sensor1_MaxStage3Pressure,Sensor1_MaxStage3PressureTime,Sensor1_SumStage3Pressure,Sensor1_CycleTime,Sensor1_SumPressure,Soft_MoldClampedCycleTime,Soft_MoldBasedCycleTime,Soft_MoldShotInterval,Soft_MoldCompleteCycleTime,Soft_DiffMaxPressure,Soft_DiffIncreasePressureTime,Soft_DiffMoldReleasePressure,OPCUA_CycleTime,OPCUA_MoldClampingTime,OPCUA_MoldReleasingTime,OPCUA_InjectionTime,OPCUA_VPChangeOverDelayTime,OPCUA_HoldingTime,OPCUA_PlastificationTime,OPCUA_VPChangeOverPosition,OPCUA_CusionVolume,OPCUA_EndHoldingPosition,OPCUA_EndPlastificationPosition,OPCUA_Nozzle1Temperature,OPCUA_Nozzle2Temperature,OPCUA_Nozzle3Temperature,OPCUA_Nozzle4Temperature,OPCUA_Nozzle5Temperature,OPCUA_IMMOilTemperature,OPCUA_FallMaterialTemperature,OPCUA_MaxInjectionPressure,OPCUA_VPChangeOverPressure,OPCUA_SumInjectionPressure,OPCUA_SumHoldingPressure,OPCUA_MaxCavityNearGatePressure,OPCUA_MaxCavityFarGatePressure,OPCUA_SumCavityPressure,OPCUA_SumMoldClampingForce,OPCUA_MaxMoldClampingForce,OPCUA_MaxBackPressure,OPCUA_DelayPlastificationTime,OPCUA_InMoldCoolingTemperature,OPCUA_OutMoldCoolingTemperature,OPCUA_MoldCoolingVolume,OPCUA_MoldReleasingPosition2,OPCUA_MoldReleasingPosition3,OPCUA_MoldReleasingPosition4,OPCUA_MoldReleasingPosition5,OPCUA_ScrewSuckBackPosition,OPCUA_ScrewRPM,OPCUA_PartWeight,OPCUA_GateClosingTime,OPCUA_MeltTemperature,OPCUA_MoldTemperature,StartShotSN,EndShotSN,GroupSN,Type
0,2254,10:29.6,12:54.0,t11,#2-4^TOY,0.05,0.066954,30.226254,0.001323,-0.000456,0,0.14,0,0.000345,0,-0.000858,0.02,0.1,1.218391,34.640692,0.002887,-0.00095,0,0.14,1.93,-0.010172,0.67,4.920607,27.99354,23.072933,0.037273,495.32,1.41,27.99354,50.007882,22.014342,-0.05133,274.74,49.957207,65.47114,15.513933,0.29,49.957207,79.62,139.891092,1205.09,139.959308,149.358556,17.110219,166.468775,-0.09,30.225909,-0.02,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2254,2254,2254,1
1,2255,13:16.0,15:40.4,t11,#2-4^TOY,0.11,1.101516,27.546706,0.003993,-0.000979,0,0.03,0.49645,44.776663,0.000447,-0.000105,0.02,0.03,0,0.000369,0,-0.000214,0,0.52,2.02,-0.010048,0.77,5.429687,27.496024,22.066337,0.038973,473.77,1.41,27.496024,50.048128,22.552104,-0.050106,269.82,49.997431,65.512633,15.515202,0.28,49.997431,79.42,139.879328,1182.87,139.949203,149.293589,17.113158,166.406747,0.08,-17.229957,-0.02,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2255,2255,2255,1
2,2256,16:02.5,18:26.4,t11,#2-4^TOY,0.05,0.544567,57.047626,0.000701,-0.000603,0,0.03,0.332434,30.836741,0.000324,-9.20E-05,0.02,0.19,3.990315,29.822788,0.006371,-0.001453,0.03,0.32,2.02,-0.010208,0.75,4.916987,27.495589,22.578602,0.038089,481.51,1.41,27.495589,50.018569,22.52298,-0.050171,273.49,49.9679,65.536865,15.568965,0.29,49.9679,78.73,139.902714,1190.77,139.974908,148.881964,17.084984,165.966948,0.02,26.210885,-0.02,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2256,2256,2256,1
3,2257,18:48.8,21:13.2,t11,#2-4^TOY,0.14,2.031487,35.601051,0.003933,-0.000959,0.04,0.02,0.603321,60.307064,0.000166,-0.000126,0.01,0.13,0,0.000366,0,-0.000858,0.01,0.29,1.99,-0.009805,

In [13]:
data3.columns = data3.iloc[0]
data3 = data3.iloc[2:, :]
data3 = data3.reset_index(drop=True)
data3.head()

,ShotSN,MoldClampedTime,MoldReleasedTime,MOID,ProdID,Sensor2_MaxPressure,Sensor2_SumStage1Pressure,Sensor2_MaxPressureTime,Sensor2_AvgStage1PressureSlope,Sensor2_AvgStage2PressureSlope,Sensor2_MoldReleasePressure,Sensor3_MaxPressure,Sensor3_SumStage1Pressure,Sensor3_MaxPressureTime,Sensor3_AvgStage1PressureSlope,Sensor3_AvgStage2PressureSlope,Sensor3_MoldReleasePressure,Sensor4_MaxPressure,Sensor4_SumStage1Pressure,Sensor4_MaxPressureTime,Sensor4_AvgStage1PressureSlope,Sensor4_AvgStage2PressureSlope,Sensor4_MoldReleasePressure,Sensor5_StartTemperature,Sensor5_MaxTemperature,Sensor5_AvgStage2TemperatureSlope,Sensor5_MoldReleaseTemperature,Sensor1_Stage1StartTime,Sensor1_Stage1EndTime,Sensor1_Stage1Period,Sensor1_AvgStage1PressureSlope,Sensor1_SumStage1Pressure,Sensor1_Stage1EndPressure,Sensor1_Stage2StartTime,Sensor1_Stage2EndTime,Sensor1_Stage2Period,Sensor1_AvgStage2PressureSlope,Sensor1_SumStage2Pressure,Sensor1_Stage3StartTime,Sensor1_Stage3EndTime,Sensor1_Stage3Period,Sensor1_MaxStage3Pressure,Sensor1_MaxStage3PressureTime,Sensor1_SumStage3Pressure,Sensor1_CycleTime,Sensor1_SumPressure,Soft_MoldClampedCycleTime,Soft_MoldBasedCycleTime,Soft_MoldShotInterval,Soft_MoldCompleteCycleTime,Soft_DiffMaxPressure,Soft_DiffIncreasePressureTime,Soft_DiffMoldReleasePressure,OPCUA_CycleTime,OPCUA_MoldClampingTime,OPCUA_MoldReleasingTime,OPCUA_InjectionTime,OPCUA_VPChangeOverDelayTime,OPCUA_HoldingTime,OPCUA_PlastificationTime,OPCUA_VPChangeOverPosition,OPCUA_CusionVolume,OPCUA_EndHoldingPosition,OPCUA_EndPlastificationPosition,OPCUA_Nozzle1Temperature,OPCUA_Nozzle2Temperature,OPCUA_Nozzle3Temperature,OPCUA_Nozzle4Temperature,OPCUA_Nozzle5Temperature,OPCUA_IMMOilTemperature,OPCUA_FallMaterialTemperature,OPCUA_MaxInjectionPressure,OPCUA_VPChangeOverPressure,OPCUA_SumInjectionPressure,OPCUA_SumHoldingPressure,OPCUA_MaxCavityNearGatePressure,OPCUA_MaxCavityFarGatePressure,OPCUA_SumCavityPressure,OPCUA_SumMoldClampingForce,OPCUA_MaxMoldClampingForce,OPCUA_MaxBackPressure,OPCUA_DelayPlastificationTime,OPCUA_InMoldCoolingTemperature,OPCUA_OutMoldCoolingTemperature,OPCUA_MoldCoolingVolume,OPCUA_MoldReleasingPosition2,OPCUA_MoldReleasingPosition3,OPCUA_MoldReleasingPosition4,OPCUA_MoldReleasingPosition5,OPCUA_ScrewSuckBackPosition,OPCUA_ScrewRPM,OPCUA_PartWeight,OPCUA_GateClosingTime,OPCUA_MeltTemperature,OPCUA_MoldTemperature,StartShotSN,EndShotSN,GroupSN,Type
0,1,26:57.0,29:21.9,t13,#2-4^TOY,1.12,20.037444,41.466042,0.02701,-0.011282,0,1.1,15.418197,37.871087,0.028518,-0.010596,0.01,1.16,7.664932,29.966163,0.038711,-0.010472,0,0.23,2.01,-0.014993,0.1,5.257923,11.830971,6.573048,0.158222,91.65,1.56,11.830971,64.48926,52.658289,-0.025827,1285.97,65.909337,66.112121,0.202784,0.25,65.960036,1.17,140.740188,1747.39,140.810355,149.948541,0,149.948541,0.02,3.594955,-0.01,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1,1,1,0
1,2,29:44.9,32:10.0,t13,#2-4^TOY,1.13,8.56581,30.725737,0.036777,-0.010257,0,1.21,14.582229,35.385415,0.033913,-0.011184,0.03,1.15,8.713303,31.384421,0.036643,-0.010501,0,0.35,2,-0.011856,0.49,4.718374,11.963373,7.244999,0.143547,98.73,1.56,11.963373,64.660281,52.696908,-0.026377,1281.03,66.080022,66.181422,0.1014,0.23,66.130726,0.65,140.89513,1744.41,140.96928,149.875703,18.203815,168.079518,-0.08,-4.659678,-0.03,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2,2,2,0
2,3,32:33.3,,t13,#2-4^TOY,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,3,3,3,0
3,4,43:43.9,46:09.0,t13,#2-4^TOY,1.13,8.963635,31.499059,0.035557,-0.010249,0.01,1.16,9.730695,31.80324,0.03616,-0.010462,0.02,1.16,7.537426,29.876897,0.038826,-0.01037,0.01,0.1,1.99,-0.010838,0.61,4.922855,11.874253,6.951398,0.148172,93.77,1.55,11.874253,64.629962,52.755709,-0.026917,1292.13,65.998388,66.150445,0.152057,0.22,66.049045,0.83,140.773559,1754.31,,,,,-0.03,-0.304181,-0.01,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,4,4,4,0
4,5,46:31.8,48:56.8,t13,#2-4^TOY,1.12,9.770712,32.114165,0.034565,-0.010204,0.01,1.24,12.882139,32.824572,0.037472,-0.0112

In [9]:
data_all = pd.concat([data1, data2])
data_all.shape

(4690, 99)

In [68]:
df = data_all[all_name]
df.shape

(4690, 31)

In [69]:
df_val = data3[all_name]
df_val.shape

(1813, 31)

In [70]:
df = df.apply(pd.to_numeric, errors='coerce')

In [71]:
df_val = df_val.apply(pd.to_numeric, errors='coerce')

In [72]:
df_nona = df.dropna()
df_nona_val = df_val.dropna()
df_nona.shape, df_nona_val.shape

((4655, 31), (1811, 31))

In [73]:
df_nona = df_nona.reset_index(drop=True)
df_nona_val = df_nona_val.reset_index(drop=True)
df_nona.head()

,Sensor2_MaxPressure,Sensor2_SumStage1Pressure,Sensor2_AvgStage1PressureSlope,Sensor2_AvgStage2PressureSlope,Sensor2_MoldReleasePressure,Sensor3_MaxPressure,Sensor3_SumStage1Pressure,Sensor3_AvgStage1PressureSlope,Sensor3_AvgStage2PressureSlope,Sensor3_MoldReleasePressure,Sensor4_MaxPressure,Sensor4_SumStage1Pressure,Sensor4_AvgStage1PressureSlope,Sensor4_AvgStage2PressureSlope,Sensor4_MoldReleasePressure,Sensor5_StartTemperature,Sensor5_MaxTemperature,Sensor5_AvgStage2TemperatureSlope,Sensor5_MoldReleaseTemperature,Sensor1_AvgStage1PressureSlope,Sensor1_SumStage1Pressure,Sensor1_Stage1EndPressure,Sensor1_AvgStage2PressureSlope,Sensor1_SumStage2Pressure,Sensor1_MaxStage3Pressure,Sensor1_SumStage3Pressure,Sensor1_SumPressure,Soft_MoldShotInterval,Soft_DiffMaxPressure,Soft_DiffMoldReleasePressure,Type
0,1.06,14.148790,0.021944,-0.011339,0.00,0.99,9.154309,0.021747,-0.010132,0.01,1.04,7.089350,0.024223,-0.010319,0.02,0.48,1.98,-0.010371,0.70,0.034745,562.98,1.44,-0.037038,898.21,0.22,65.09,1821.66,0.000000,0.07,-0.01,0
1,0.96,8.488392,0.020756,-0.010049,0.00,1.12,10.489044,0.025541,-0.010781,0.06,0.99,10.312204,0.021196,-0.010413,0.00,0.54,1.98,-0.010136,0.73,0.030567,644.65,1.43,-0.041024,822.68,0.23,64.68,1825.97,17.399743,-0.16,-0.06,0
2,0.98,16.411448,0.018318,-0.011088,0.00,1.02,7.885406,0.020841,-0.010282,0.02,1.09,11.335504,0.023595,-0.011182,0.02,0.43,1.98,-0.013864,0.27,0.030523,650.36,1.43,-0.035866,808.26,1.12,2.52,1820.92,17.396936,-0.04,-0.02,0
3,1.09,10.686955,0.024935,-0.010695,0.04,1.03,7.681343,0.023598,-0.010294,0.01,1.02,11.518071,0.020175,-0.010873,0.00,0.27,1.98,-0.012057,0.49,0.036441,553.08,1.47,-0.036021,912.16,1.07,1.93,1823.19,17.509495,0.06,0.03,0
4,1.00,13.724422,0.018457,-0.010940,0.00,1.08,9.174600,0.024435,-0.010612,0.04,1.09,8.964627,0.025036,-0.010788,0.03,0.48,1.98,-0.010695,0.66,0.035590,560.35,1.46,-0.037569,901.33,0.23,65.12,1821.57,17.661424,-0.08,-0.04,0


In [74]:
X = df_nona.iloc[:, :-1]
y = df_nona['Type']
X.shape, y.shape

((4655, 30), (4655,))

In [76]:
X1 = df_nona_val.iloc[:, :-1]
y1 = df_nona_val['Type']
X1.shape, y1.shape

((1811, 30), (1811,))

In [78]:
y.unique()

array([0, 1])

In [79]:
y[y==1] = -1  # 将异常点标记为-1
y[y==0] = 1   # 将正常点标记为1

In [81]:
y1[y1==0] = 1   # 将正常点标记为1

In [83]:
y.unique()

array([ 1, -1])

In [82]:
y1.unique()

array([1])

In [24]:
y.tail()

4650   -1
4651   -1
4652   -1
4653   -1
4654   -1
Name: Type, dtype: int64

In [84]:
scaler = StandardScaler()

In [26]:
# X = scaler.fit_transform(X)

In [37]:
# X1 = scaler.transform(X1)

In [85]:
# 划分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [86]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, X1.shape, y1.shape

((3258, 30), (1397, 30), (3258,), (1397,), (1811, 30), (1811,))

In [87]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X1)

In [88]:
X_positive = X_train[y_train == 1]

In [89]:
X_positive.shape

(3177, 30)

In [90]:
clf = IsolationForest(n_estimators=85, contamination=0.04, random_state=42)

In [91]:
clf.fit(X_positive)

IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.04,
                max_features=1.0, max_samples='auto', n_estimators=85,
                n_jobs=None, random_state=42, verbose=0, warm_start=False)

In [92]:
y_pred = clf.predict(X_test)

In [93]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.946


In [96]:
confusion_matrix(y_test, y_pred)

array([[  20,   12],
       [  64, 1301]])

In [94]:
y_pred_val = clf.predict(X_val)

In [95]:
accuracy = accuracy_score(y1, y_pred_val)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.866


In [97]:
confusion_matrix(y1, y_pred_val)

array([[   0,    0],
       [ 243, 1568]])

In [130]:
# X[y == 1].shape

(4542, 30)

In [232]:
# from sklearn.ensemble import IsolationForest
# from sklearn.model_selection import cross_val_score

# # 将所有标签为 1 的样本视为正样本
# X = df_nona.iloc[:, :-1]
# y = df_nona['Type']

# y[y==1] = -1  # 将异常点标记为-1
# y[y==0] = 1   # 将正常点标记为1

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# X_positive = X_train[y_train == 1]

# # param_grid = {'n_estimators': [10, 50, 100, 500]}

# clf = IsolationForest(n_estimators=500, random_state=42)

# # cv_results = cross_val_score(clf, X_positive, cv=5, scoring='accuracy', fit_params=param_grid)

# # print("Best n_estimators:", param_grid['n_estimators'][cv_results.argmax()])

# clf.fit(X_positive)

# y_pred = clf.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)

# print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.940


In [233]:
# confusion_matrix(y_test, y_pred)

array([[ 13,  10],
       [ 46, 862]])

# 不同的算法
# 目前KO榜TOP 1為田弘光(誤)IsolationForest,其次為OneSVM
# 以這兩種下去調整參數

# 1. One SVM算法

In [34]:
# 创建One-Class SVM模型并拟合训练数据
ocsvm = OneClassSVM(kernel='rbf', nu=0.1)
ocsvm.fit(X_train)

# 使用测试集评估模型性能
y_pred = ocsvm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=-1)
recall = recall_score(y_test, y_pred, pos_label=-1)
f1 = f1_score(y_test, y_pred, pos_label=-1)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1: {f1:.3f}")
print(f'MAE: {mae(y_test, y_pred)}')
print(f'MSE: {mse(y_test, y_pred)}')
print(f'RMSE: {sqrt(mse(y_test, y_pred))}')
print(f'R2: {r2_score(y_test, y_pred)}')


Accuracy: 0.918
Precision: 0.216
Recall: 0.731
F1: 0.333
MAE: 0.16326530612244897
MSE: 0.32653061224489793
RMSE: 0.5714285714285714
R2: -2.0070548236294097


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 19,   7],
       [ 69, 836]])

In [21]:
'''
            预测为正类   预测为负类
实际为正类       TP        FN
实际为负类       FP        TN

'''

# 2. DeepOCC算法 (無法判斷出不良品)

In [75]:
# 将数据转换为PyTorch张量
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train.to_numpy())
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [76]:
# 定义DeepOCC模型
class DeepOCC(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepOCC, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out

In [80]:
# 定义DeepOCC模型的参数
input_dim = 30
hidden_dim = 10
output_dim = 1

# 初始化DeepOCC模型
deep_occ = DeepOCC(input_dim, hidden_dim, output_dim)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(deep_occ.parameters(), lr=0.01)

# 训练DeepOCC模型
for epoch in range(50):
    for x, y in train_loader:
        optimizer.zero_grad()
        output = deep_occ(x)
        loss = criterion(output, torch.ones(output.shape))
        loss.backward()
        optimizer.step()

# 对测试集进行分类
X_test_tensor = torch.Tensor(X_test)
test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32)
y_pred = []
for x in test_loader:
    output = deep_occ(x[0])
    y_pred.extend(output.detach().numpy())

# 将预测结果转换为二元分类结果
y_pred = [1 if x > 0.5 else -1 for x in y_pred]

# 计算模型的准确率
accuracy = sum([1 if x == y else -1 for x, y in zip(y_test, y_pred)]) / len(y_test)
precision = precision_score(y_test, y_pred, pos_label=-1)
recall = recall_score(y_test, y_pred, pos_label=-1)
f1 = f1_score(y_test, y_pred, pos_label=-1)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1: {f1:.3f}")
print(f'MAE: {mae(y_test, y_pred)}')
print(f'MSE: {mse(y_test, y_pred)}')
print(f'RMSE: {sqrt(mse(y_test, y_pred))}')
print(f'R2: {r2_score(y_test, y_pred)}')

Accuracy: 0.942
Precision: 0.000
Recall: 0.000
F1: 0.000
MAE: 0.05800214822771214
MSE: 0.11600429645542427
RMSE: 0.3405940346738684
R2: -0.06829579260518504


In [81]:
confusion_matrix(y_test, y_pred)

array([[  0,  26],
       [  1, 904]])

# 3. IsolationForest算法

In [49]:
from sklearn.ensemble import IsolationForest

# 将所有标签为 1 的样本视为正样本
X_positive = X_train[y_train == 1]

# X = scaler.fit_transform(X_positive)

clf = IsolationForest(n_estimators=85, contamination=0.04, random_state=42)

clf.fit(X_positive)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.3f}")

ValueError: Found array with 0 sample(s) (shape=(0, 30)) while a minimum of 1 is required.

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  20,   12],
       [  64, 1301]])

In [38]:
y_pred_val = clf.predict(X1)

accuracy_val = accuracy_score(y1, y_pred_val)

print(f"Accuracy: {accuracy_val:.3f}")

Accuracy: 0.000


In [39]:
confusion_matrix(y1, y_pred_val)

array([[   0,    0],
       [1811,    0]])

In [317]:
# 原結果
# Accuracy: 0.951
# Precision: 0.308
# Recall: 0.615
# F1: 0.410
# MAE: 0.09881847475832438
# MSE: 0.19763694951664876
# RMSE: 0.44456377440885664
# R2: -0.8200594985125376
# array([[ 16,  10],
#        [ 36, 869]])

In [25]:
# 计算混淆矩阵
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# 输出评估指标
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1_score)
print(f'R2: {r2_score(y_test, y_pred)}')

Accuracy: 0.945597709377237
Precision: 0.9908606245239908
Recall: 0.9531135531135531
F1 Score: 0.9716206123973113
R2: -1.430677655677656


# 4. Autoencoder算法

In [19]:
import numpy as np
from tensorflow import keras
from sklearn.datasets import make_classification

# 将所有标签为 1 的样本视为正样本
X_positive = X_train[y_train == 1]

# 定义 Autoencoder 模型
input_shape = (X_positive.shape[1],)
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=input_shape),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(input_shape[0], activation='linear')
])

# 编译模型
model.compile(optimizer='adam', loss='mse')

# 使用 Autoencoder 进行拟合
model.fit(X_positive, X_positive, epochs=100, batch_size=8)

# 預測模型
y_pred = model.predict(X_test)

# 计算重构误差
reconstruction_errors = np.mean(np.square(X_test - y_pred), axis=1)

# 定义阈值，将重构误差小于阈值的样本视为正常样本，大于阈值的样本视为异常样本
threshold = 0.1
y_pred = np.where(reconstruction_errors < threshold, 1, -1)

# 打印预测结果
# print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=-1)
recall = recall_score(y_test, y_pred, pos_label=-1)
f1 = f1_score(y_test, y_pred, pos_label=-1)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1: {f1:.3f}")
print(f'MAE: {mae(y_test, y_pred)}')
print(f'MSE: {mse(y_test, y_pred)}')
print(f'RMSE: {sqrt(mse(y_test, y_pred))}')
print(f'R2: {r2_score(y_test, y_pred)}')

2023-05-03 10:18:13.993725: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 10:18:15.918240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10041 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1


Epoch 1/100
398/398 [==============================] - 1s 2ms/step - loss: 0.7864
Epoch 2/100
398/398 [==============================] - 1s 2ms/step - loss: 0.5873
Epoch 3/100
398/398 [==============================] - 1s 2ms/step - loss: 0.5124
Epoch 4/100
398/398 [==============================] - 1s 2ms/step - loss: 0.4610
Epoch 5/100
398/398 [==============================] - 1s 2ms/step - loss: 0.4265
Epoch 6/100
398/398 [==============================] - 1s 2ms/step - loss: 0.4025
Epoch 7/100
398/398 [==============================] - 1s 2ms/step - loss: 0.3819
Epoch 8/100
398/398 [==============================] - 1s 2ms/step - loss: 0.3629
Epoch 9/100
398/398 [==============================] - 1s 2ms/step - loss: 0.3342
Epoch 10/100
398/398 [==============================] - 1s 2ms/step - loss: 0.3214
Epoch 11/100
398/398 [==============================] - 1s 2ms/step - loss: 0.3086
Epoch 12/100
398/398 [==============================] - 1s 2ms/step - loss: 0.2928
Epoch 13/100


In [104]:
import numpy as np
from tensorflow import keras
from sklearn.datasets import make_classification

# 将所有标签为 1 的样本视为正样本
X_positive = X[y == 1]

# 定义 Autoencoder 模型
input_shape = (X_positive.shape[1],)
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=input_shape),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(input_shape[0], activation='linear')
])

# 编译模型
model.compile(optimizer='adam', loss='mse')

# 使用 Autoencoder 进行拟合
model.fit(X_positive, X_positive, epochs=100, batch_size=8)

# 預測模型
y_pred = model.predict(X_test)

# 计算重构误差
reconstruction_errors = np.mean(np.square(X_test - y_pred), axis=1)

# 定义阈值，将重构误差小于阈值的样本视为正常样本，大于阈值的样本视为异常样本
threshold = 0.1
y_pred = np.where(reconstruction_errors < threshold, 1, -1)

# 打印预测结果
# print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=-1)
recall = recall_score(y_test, y_pred, pos_label=-1)
f1 = f1_score(y_test, y_pred, pos_label=-1)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1: {f1:.3f}")
print(f'MAE: {mae(y_test, y_pred)}')
print(f'MSE: {mse(y_test, y_pred)}')
print(f'RMSE: {sqrt(mse(y_test, y_pred))}')
print(f'R2: {r2_score(y_test, y_pred)}')

Epoch 1/100
568/568 [==============================] - 1s 2ms/step - loss: 0.7558
Epoch 2/100
568/568 [==============================] - 1s 2ms/step - loss: 0.5442
Epoch 3/100
568/568 [==============================] - 1s 2ms/step - loss: 0.4795
Epoch 4/100
568/568 [==============================] - 1s 2ms/step - loss: 0.4284
Epoch 5/100
568/568 [==============================] - 1s 2ms/step - loss: 0.3919
Epoch 6/100
568/568 [==============================] - 1s 2ms/step - loss: 0.3723
Epoch 7/100
568/568 [==============================] - 1s 2ms/step - loss: 0.3525
Epoch 8/100
568/568 [==============================] - 1s 2ms/step - loss: 0.3294
Epoch 9/100
568/568 [==============================] - 1s 2ms/step - loss: 0.3157
Epoch 10/100
568/568 [==============================] - 1s 2ms/step - loss: 0.3026
Epoch 11/100
568/568 [==============================] - 1s 2ms/step - loss: 0.2974
Epoch 12/100
568/568 [==============================] - 1s 2ms/step - loss: 0.2873
Epoch 13/100


In [105]:
confusion_matrix(y_test, y_pred)

array([[ 26,   0],
       [108, 797]])

# 5. Deep Support Vector Data Description算法 (無法判斷不良品)

In [20]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report

# # 将所有标签为 1 的样本视为正样本
# X_positive = X[y == 1]

# # 对正样本进行标准化处理
# scaler = StandardScaler()
# X_positive = scaler.fit_transform(X_positive)

# 划分训练集和测试集
# X_train, X_test = train_test_split(X_positive, test_size=0.2, random_state=42)

# 定义 Deep SVDD 模型
input_shape = (X_positive.shape[1],)
model = Sequential([
    Dense(64, activation='relu', input_shape=input_shape, kernel_regularizer=regularizers.l2(0.01)),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(1, activation='sigmoid')
])

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')

# 使用 Deep SVDD 进行拟合
model.fit(X_train, np.ones(X_train.shape[0]), epochs=50, batch_size=32, verbose=0)

# 对测试集进行预测，并计算性能指标
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, -1)

cm = confusion_matrix(np.ones(X_test.shape[0]), y_pred)
print('Confusion matrix:\n', cm)

report = classification_report(np.ones(X_test.shape[0]), y_pred)
print('Classification report:\n', report)


Confusion matrix:
 [[1397]]
Classification report:
               precision    recall  f1-score   support

         1.0       1.00      1.00      1.00      1397

    accuracy                           1.00      1397
   macro avg       1.00      1.00      1.00      1397
weighted avg       1.00      1.00      1.00      1397



In [21]:
confusion_matrix(y_test, y_pred)

array([[   0,   32],
       [   0, 1365]])

In [98]:
# list(zip(y_test, y_pred))